## Import datas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, learning_curve, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from scipy import stats



import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter("ignore")

In [2]:
# Use pandas to loadinto a DataFrame
# Y1.csv doesn’t have a header so
# add one when loading the file
X1 = pd.read_csv("data/X1.csv")
Y1 = pd.read_csv("data/Y1.csv", header=None, names=['revenue'])


# ENLEVER colonne "Unnamed" du dataset : utilité ? -> Pas listé dans les features du pdf
X = X1.drop(['Unnamed: 0', 'img_url', 'description'], axis=1)

### Pre data visualisation

In [3]:
"""
print(X.describe())

plt.figure(figsize=(7,4))
corr = X.corr(numeric_only=True)
heatmap = sns.heatmap(corr, annot=True, cmap="coolwarm", fmt='.1g')
"""

'\nprint(X.describe())\n\nplt.figure(figsize=(7,4))\ncorr = X.corr(numeric_only=True)\nheatmap = sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=\'.1g\')\n'

## Preprocessing the data
### Preprocessing functions

In [4]:
"""
Creating empty DataFrame to start
"""
n_samples = X.shape[0]
data = pd.DataFrame()

In [5]:
"""
Keeping the directly usable features
"""
def get_directrly_usable_features(df):
    directly_usable_features = ["ratings", "n_votes", "is_adult"]
    for feature in directly_usable_features:
        df[feature] = X[feature]
    return df

In [6]:
"""
Dealing with the "production_year" feature
"""

def get_prod_year_feature(df, params):
    
    style = params["production_year_style"] # "per_quantile" / "per_period_length" / "no_period"
    if style != "no_period" :
        n_year_period = params["n_year_period"]

    # Removing previously computed categorie(s) for the "production_year" initial feature

    for feature in df.columns:
        if len(feature) >= 8 and (feature[:6] == "period" or feature == "production_year"):
            df.drop(feature, axis=1, inplace=True)

    # Creating new categorie(s) for the "production_year" initial feature

    prod_year = X["production_year"].copy()
    if style == "per_quantile" or style == "per_period_length":
        categories = np.ones((n_year_period, n_samples))

        if style == "per_quantile":
            thresholds = prod_year.quantile(np.arange(1, n_year_period) / n_year_period)
        else :
            thresholds = np.min(prod_year) + (np.max(prod_year) - np.min(prod_year))*np.arange(1, n_year_period)/n_year_period
        for i, threshold in enumerate(thresholds):
            categories[i+1] = (prod_year >= threshold).astype(int)
            categories[i] -= categories[i+1]
        for period in range(n_year_period):
            df["period {}".format(period)] = categories[period]
    elif style == "no_period":
        df["production_year"] = prod_year
    return df

In [7]:
"""
Dealing with the "runtime" feature

The problem is here that we have some missing values, we have to deal with it.
"""

def get_runtime_feature(df, params):

    # Add other smarter ways ?
    
    replace_type = params["runtime_replace_type"] # "zero" / "mean"
    
    runtime = X["runtime"].copy()
    if replace_type == "zero":
        runtime[runtime == "\\N"] = 0
    if replace_type == "mean":
        mean = np.mean(runtime[runtime != "\\N"].astype(float))
        runtime[runtime == "\\N"] = mean
    df["runtime"] = runtime.astype(float)    
    return df

In [8]:
import copy
"""
Dealing with the "studio" feature


Juste rajouter toute les features une par une me semblait un peu lourd (yen a 509), ducoup j'effectue PCA dessus.
Jsp si ça se fait ? (on peut changer l'algo de dimensionality reduction aussi si on veut)
"""

def get_studio_feature(df, params):
    use_PCA = params["studio_use_PCA"]
    if use_PCA :
        dim = params["studio_PCA_dim"]

    # Removing previously computed categorie(s) for the "studio" initial feature
    for feature in df.columns:
        if len(feature) >= 10 and feature[:10] == "studio_PC_":
            df.drop(feature, axis=1, inplace=True)

    # Creating new categorie(s) for the "studio" initial feature
    studio = copy.deepcopy(X["studio"])
    studio_labels = np.unique(studio)
    studio_features = np.zeros((len(studio_labels), n_samples))
    for i, label in enumerate(studio_labels) :
        studio_features[i] = (studio == label).astype(int)
    
    # Applying pca or not
    if use_PCA :
        pca = PCA(n_components=dim)
        out = pca.fit_transform(studio_features.T)
    else :
        normals = studio_features[np.count_nonzero(studio_features, axis=1) > 5].T
        outliers = np.sum(studio_features[np.count_nonzero(studio_features, axis=1) <= 5].T, axis = 1)
        out = np.zeros((normals.shape[0], normals.shape[1] + 1))
        out[:, :-1] = normals
        out[:, -1] = outliers
        dim = out.shape[1]
    
    df[["studio_PC_{}".format(i) for i in range(dim)]] = out
    return df

# Ya plein de warnings quand dim trop grand ou pas de PCA /: 
# jsp comment regler ça... en utilisant pd.concat ça tourne vraiment extrêmement lentement

In [9]:
"""
Dealing with the "genres" feature

Je rajoute juste une feature par genre, j'espère ça suffit ? -> pt-être dimension reduction sur ça aussi ?
Certains films ont pas de genre attitré (genre = "\\N"),
ce que j'ai fait ici considère ça comme un genre à part entière, ptet on peut deal le truc autrement ?
"""

def get_genre_feature(df):

    X.loc[X["genres"] == "\\N", "genres"] = "Others"
    all_genres = X["genres"].copy()
    diff_genres = []

    for genres in np.unique(all_genres):
        for genre in genres.split(",") :
            if not genre in diff_genres :
                diff_genres.append(genre)

    for genre in diff_genres:
        df[genre] = [1 if genre in genres.split(",") else 0 for genres in all_genres]  
    return df

In [10]:
"""
Dealing with the "text_embeddings" feature


Dimension of embedding space is too high -> dimensionnality reduction
J'utilise que PCA pr l'instant ici aussi
"""
def get_text_embedding_feature(df, params):
    
    output_dim = params["text_embedding_PCA_dim"] # output dimension of PCA

    # Removing previously computed categorie(s) for the "text_embedding" initial feature
    for feature in df.columns:
        if len(feature) >= 18 and feature[:18] == "text_embedding_PC_":
            df.drop(feature, axis=1, inplace=True)

    # Creating new categorie(s) for the "text_embedding" initial feature
    text_embeddings = X["text_embeddings"]
    input_dim = 768
    embeddings = np.zeros((n_samples, input_dim))
    for i, text_embedding in enumerate(text_embeddings):
        embeddings[i] = list(map(float,text_embedding[1:-1].split(",")))

    # applying PCA
    pca = PCA(n_components=output_dim)
    output = pca.fit_transform(embeddings)

    df[["text_embedding_PC_{}".format(i) for i in range(output_dim)]] = output
    return df

In [11]:
"""
Dealing with the "img_embeddings" feature


Dimension of embedding space is too high -> dimensionnality reduction
J'utilise que PCA pr l'instant ici aussi
"""
def get_img_embedding_feature(df, params):
    output_dim = params["img_embedding_PCA_dim"] # output dimension of PCA

    # Removing previously computed categorie(s) for the "img_embedding" initial feature
    for feature in df.columns:
        if len(feature) >= 17 and feature[:17] == "img_embedding_PC_":
            df.drop(feature, axis=1, inplace=True)

    # Creating new categorie(s) for the "img_embedding" initial feature
    img_embeddings = X["img_embeddings"]
    input_dim = 2048
    embeddings = np.zeros((n_samples, input_dim))
    for i, img_embedding in enumerate(img_embeddings):
        embeddings[i] = list(map(float,img_embedding[1:-1].split(",")))

    # applying PCA
    pca = PCA(n_components=output_dim)
    output = pca.fit_transform(embeddings)

    df[["img_embedding_PC_{}".format(i) for i in range(output_dim)]] = output
    return df

In [12]:
def create_preprocessed(params):
    df = pd.DataFrame()
    df = get_genre_feature(df)
    df = get_directrly_usable_features(df)
    df = get_prod_year_feature(df, params)
    df = get_studio_feature(df, params)
    df = get_runtime_feature(df, params)
    df = get_text_embedding_feature(df, params)
    df = get_img_embedding_feature(df, params)
    return df

In [13]:
def create_preprocessed_dict(params):
    final = {}
    year_dict = {}
    for val1 in params["production_year_style"] :
        for val2 in params["n_year_period"] :
            tmp = {}
            tmp["production_year_style"] = val1
            tmp["n_year_period"] = val2
            year_dict[val1+str(val2)] = get_prod_year_feature(pd.DataFrame(), tmp)
    final["year"] = year_dict
    
    studio_dict = {}
    for val1 in params["studio_use_PCA"] :
        for val2 in params["studio_PCA_dim"] :
            tmp = {}
            tmp["studio_use_PCA"] = val1
            tmp["studio_PCA_dim"] = val2
            studio_dict[str(val1)+str(val2)] = get_studio_feature(pd.DataFrame(), tmp)
    final["studio"] = studio_dict
    
    runtime_dict = {}
    for val in params["runtime_replace_type"] :
        tmp = {}
        tmp["runtime_replace_type"] = val
        runtime_dict[val] = get_runtime_feature(pd.DataFrame(), tmp)
    final["runtime"] = runtime_dict
    
    text_dict = {}
    for val in params["text_embedding_PCA_dim"] :
        tmp = {}
        tmp["text_embedding_PCA_dim"] = val
        text_dict[val] = get_text_embedding_feature(pd.DataFrame(), tmp)
    final["text"] = text_dict
    
    img_dict = {}
    for val in params["img_embedding_PCA_dim"] :
        tmp = {}
        tmp["img_embedding_PCA_dim"] = val
        img_dict[val] = get_img_embedding_feature(pd.DataFrame(), tmp)
    final["img"] = img_dict   
    final["genre"] = get_genre_feature(pd.DataFrame())
    final["base"] = get_directrly_usable_features(pd.DataFrame())
    return final

In [14]:
def create_preprocessed_from_dict(params_dict, params):
    frames = []
    frames.append(params_dict["genre"])
    frames.append(params_dict["base"])
    frames.append(params_dict["year"][params["production_year_style"] + str(params["n_year_period"])])
    frames.append(params_dict["studio"][str(params["studio_use_PCA"]) + str(params["studio_PCA_dim"])])
    frames.append(params_dict["runtime"][params["runtime_replace_type"]])
    frames.append(params_dict["text"][params["text_embedding_PCA_dim"]])
    frames.append(params_dict["img"][params["img_embedding_PCA_dim"]])
    return pd.concat(frames, axis=1)

### Preprocessing execution

In [15]:
"""
Example of data preprocessing
"""

params = {
    "production_year_style" : "per_quantile", # "per_quantile" / "per_period_length" / "no_period"
    "n_year_period" : 5,
    "runtime_replace_type" : "mean", # "mean" / "zero"
    "studio_use_PCA" : True,
    "studio_PCA_dim" : 50,
    "text_embedding_PCA_dim" : 50,
    "img_embedding_PCA_dim" : 50
}

preprocessed_data = create_preprocessed(params)

In [16]:
all_params = {
    "production_year_style" : ["per_quantile", "per_period_length", "no_period"], # "per_quantile" / "per_period_length" / "no_period"
    "n_year_period" : [3, 5, 10],
    "runtime_replace_type" : ["mean", "zero"], # "mean" / "zero"
    "studio_use_PCA" : [True],
    "studio_PCA_dim" : [1, 10],
    "text_embedding_PCA_dim" : [1, 10],
    "img_embedding_PCA_dim" : [1, 10]
}

params = {
    "production_year_style" : "per_quantile", # "per_quantile" / "per_period_length" / "no_period"
    "n_year_period" : 5,
    "runtime_replace_type" : "mean", # "mean" / "zero"
    "studio_use_PCA" : True,
    "studio_PCA_dim" : 1,
    "text_embedding_PCA_dim" : 10,
    "img_embedding_PCA_dim" : 10
}

preprocessed_dict = create_preprocessed_dict(all_params)
preprocessed_data = create_preprocessed_from_dict(preprocessed_dict, params)
preprocessed_data_2 = create_preprocessed(params)

In [17]:
preprocessed_data.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,img_embedding_PC_0,img_embedding_PC_1,img_embedding_PC_2,img_embedding_PC_3,img_embedding_PC_4,img_embedding_PC_5,img_embedding_PC_6,img_embedding_PC_7,img_embedding_PC_8,img_embedding_PC_9
0,0,1,0,1,0,0,1,0,0,0,...,-3.256436,-6.489770,-3.349808,-1.176245,5.893610,-0.582044,-1.161454,2.123083,-5.272238,0.114909
1,1,0,0,0,1,0,1,0,0,0,...,-5.427690,-1.379267,-3.140826,-1.040603,-7.670217,4.403435,1.477009,-4.139292,3.953297,-0.370849
2,0,0,0,0,0,0,1,1,0,0,...,2.052495,4.428169,-0.752572,-4.224751,0.135466,-6.221801,-0.616367,-2.482779,-0.894261,1.929961
3,0,0,0,0,0,0,1,0,0,0,...,-0.471225,-1.694614,-0.049679,1.635100,4.501932,-1.861986,1.262274,3.071732,0.000716,0.814752
4,0,0,0,0,1,0,1,0,0,0,...,-6.992412,-0.808753,2.917086,-3.761091,-0.365959,2.251730,0.507456,-2.131749,0.284576,0.237786


In [18]:
preprocessed_data_2.head()

,Action,Adventure,Animation,Comedy,Crime,Documentary,Drama,Family,Fantasy,History,...,img_embedding_PC_0,img_embedding_PC_1,img_embedding_PC_2,img_embedding_PC_3,img_embedding_PC_4,img_embedding_PC_5,img_embedding_PC_6,img_embedding_PC_7,img_embedding_PC_8,img_embedding_PC_9
0,0,1,0,1,0,0,1,0,0,0,...,-3.256434,-6.489767,-3.349805,-1.176368,5.894304,-0.583349,-1.162693,2.124027,-5.306920,0.080211
1,1,0,0,0,1,0,1,0,0,0,...,-5.427684,-1.379301,-3.140865,-1.040555,-7.670521,4.405017,1.473135,-4.142650,3.978261,-0.383656
2,0,0,0,0,0,0,1,1,0,0,...,2.052491,4.428200,-0.752544,-4.224905,0.135836,-6.222792,-0.613739,-2.478127,-0.912046,1.931055
3,0,0,0,0,0,0,1,0,0,0,...,-0.471226,-1.694664,-0.049705,1.635398,4.501224,-1.859931,1.256131,3.068922,0.013728,0.779192
4,0,0,0,0,1,0,1,0,0,0,...,-6.992419,-0.808740,2.917089,-3.761585,-0.365662,2.248980,0.514345,-2.134935,0.280256,0.262328


### Data visualisation

In [19]:
"""
plt.figure(figsize=(12,8))
corr = preprocessed_data.corr()

heatmap = sns.heatmap(corr, cmap="coolwarm"
"""

'\nplt.figure(figsize=(12,8))\ncorr = preprocessed_data.corr()\n\nheatmap = sns.heatmap(corr, cmap="coolwarm"\n'

### Feature Selection


In [20]:
# Creer un data training/validation splités a partir du X1 (on garde X2 pour les vrais tests) 
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data, Y1, test_size = 0.2, random_state=0)

In [21]:
# Score computation : Root Mean Square Error

def compute_rmse(predict, target):
    return mean_squared_error(predict, target, squared=False)

def compute_rmse2(predict, target):
    if len(target.shape) == 2:
        target = target.squeeze()
    if len(predict.shape) == 2:
        predict = predict.squeeze()
    diff = target - predict
    if len(diff.shape) == 1:
        diff = np.expand_dims(diff, axis=-1)
    rmse = np.sqrt(diff.T@diff / diff.shape[0])
    return float(rmse)

custom_scorer = make_scorer(compute_rmse, greater_is_better=False)

In [22]:
def perform_rand_search(model_, params, scoring, X_train, y_train):
    grid = RandomizedSearchCV(model_, params, cv=5, scoring=scoring, n_jobs=-1)
    grid.fit(X_train,y_train)
    print("grid.best_score_: ", grid.best_score_)
    print("grid.best_params_: ",grid.best_params_)
    return grid
 

In [23]:
def perform_grid_search(model_, params, scoring, X_train, y_train):

    grid = GridSearchCV(model_, params, cv=5, scoring=scoring, n_jobs=-1)
    grid.fit(X_train,y_train)
    print("grid.best_score_: ", grid.best_score_)
    print("grid.best_params_: ",grid.best_params_)
    return grid
 

In [24]:
scores = mutual_info_regression(X_train, np.ravel(y_train))

### Linear Regression

In [25]:
model = LinearRegression(n_jobs=-1)
model.fit(X_train, y_train)
model.predict(X_test)

# Évaluez le modèle en utilisant le scoreur personnalisé
score = custom_scorer(model, X_test, y_test)
print("RMSE Score: %.2f$" % score)
print("Score: %.2f%%" %model.score(X_test, y_test))

# Évaluez le modèle en utilisant le cross_val
rmse_score = cross_val_score(LinearRegression(), preprocessed_data, Y1.values.ravel(), cv=5, scoring=custom_scorer)
linear_regressor_score = cross_val_score(LinearRegression(), preprocessed_data, Y1.values.ravel(), cv=5)
print("\nCV RMSE Score : %.2f$" % np.mean(rmse_score))
print("CV Score : %.2f%%" % np.mean(linear_regressor_score))

RMSE Score: -55366574.34$
Score: 0.42%

CV RMSE Score : -59988453.18$
CV Score : 0.32%


In [34]:
import copy



class Solution:
    def __init__(self, params, score):
        self.params = params
        self.score = score

class LocalSearch :
    
    def __init__(self, regressor, params):
        self.regressor = regressor
        self.params = params
        self.preprocessed_dict = create_preprocessed_dict(params)
        self.visited = []
    
    def clean(self, params):
        copied = copy.deepcopy(params)
        if copied["production_year_style"] == "no_period":
            copied.pop("n_year_period")
        if copied["studio_use_PCA"] == False:
            copied.pop("studio_PCA_dim")
        return copied
    
    def is_visited(self, params):
        return str(self.clean(params)) in self.visited
    
    def visit(self, params):
        self.visited.append(str(self.clean(params)))
    
    def get_solution(self, params):
        preprocessed_data = copy.deepcopy(create_preprocessed_from_dict(self.preprocessed_dict, params))
        # current_score = np.mean(cross_val_score(regressor, preprocessed_data, Y1, cv=5, scoring='neg_root_mean_squared_error'))
        current_score = np.mean(cross_val_score(regressor, preprocessed_data, Y1, cv=5))
        self.visit(params)
        return Solution(params, current_score)
    
    def initiate(self):
        current_params = {}
        for param in self.params.keys():
            current_params[param] = np.random.choice(self.params[param])
        self.current_solution = self.get_solution(current_params)
        self.best_solution = copy.deepcopy(self.current_solution)
    
    def transitions(self):
        neighbors = []
        current_params = self.current_solution.params
        for param in self.params.keys():
            copied = copy.deepcopy(self.params[param])
            np.random.shuffle(copied)
            for value in copied:
                new_params = copy.deepcopy(current_params)
                new_params[param] = value
                if not self.is_visited(new_params):
                    neighbors.append(new_params)
                    break
        return neighbors
    
    def choose(self, neighbors, n):
        if len(neighbors) <= n :
            return neighbors
        return np.random.choice(neighbors, size=n, replace = False)
    
    def get_difference(self, other):
        first = self.current_solution.params
        for param in first.keys():
            if first[param] != other[param]:
                return param, first[param], other[param]
        return None
    
    def compute(self):
        self.initiate()
        print("INITIALISATION ENDED : initial score of {:.2%}".format(self.best_solution.score))
        no_upgrade = 0;
        while no_upgrade <= 30:
            neighbors = self.transitions()
            if len(neighbors) == 0:
                break
            neighbors = self.choose(neighbors, 3)
            curr_best_sol = Solution(None, -float("inf"))
            for neighbor in neighbors :
                solution = self.get_solution(neighbor)
                if solution.score > curr_best_sol.score :
                    curr_best_sol = solution
                diff = self.get_difference(neighbor)
                # print("{} : {} -> {} score {:.2%}".format(diff[0], diff[1], diff[2],solution.score))
            self.current_solution = copy.deepcopy(curr_best_sol)
            # print("updated current")
            if curr_best_sol.score >= self.best_solution.score:
                self.best_solution = copy.deepcopy(curr_best_sol)
                print("SOLUTION UPGRADED : new score of {:.2%}".format(self.best_solution.score))
                no_upgrade = 0
            no_upgrade += 1
        
params = {
    "production_year_style" : ["per_quantile", "per_period_length", "no_period"], # "per_quantile" / "per_period_length" / "no_period"
    "n_year_period" : [3, 5, 10],
    "runtime_replace_type" : ["mean", "zero"], # "mean" / "zero"
    "studio_use_PCA" : [False, True],
    "studio_PCA_dim" : [1, 10],
    "text_embedding_PCA_dim" : [1, 10, 20, 50, 100],
    "img_embedding_PCA_dim" : [1, 10, 20, 50, 100]
}

regressor = LinearRegression(copy_X = True, n_jobs=-1)
localsearch = LocalSearch(regressor, params)
localsearch.compute()

INITIALISATION ENDED : initial score of 33.74%
SOLUTION UPGRADED : new score of 39.59%
SOLUTION UPGRADED : new score of 39.89%
SOLUTION UPGRADED : new score of 40.01%
SOLUTION UPGRADED : new score of 40.06%
SOLUTION UPGRADED : new score of 40.12%
SOLUTION UPGRADED : new score of 40.15%
SOLUTION UPGRADED : new score of 40.18%


In [39]:
print(localsearch.best_solution.score)
print(localsearch.best_solution.params)

0.40184168966812434
{'production_year_style': 'per_quantile', 'n_year_period': 10, 'runtime_replace_type': 'zero', 'studio_use_PCA': False, 'studio_PCA_dim': 1, 'text_embedding_PCA_dim': 10, 'img_embedding_PCA_dim': 1}


### GridSearch on RandomForestRegressor

In [28]:
## GridSearch
# Définir les valeurs à tester pour les hyperparamètres du modèle
params =    {"n_estimators": [500], 
            "criterion": ["squared_error"], # ["squared_error", "absolute_error", "poisson"],
            "max_depth": [1],               # [None, 1, 2, 3],
            "min_samples_split": [1.0],       # [1, 2, 3],
            "max_features" : ["sqrt"],      # ["sqrt", "log2", None]
            }
rfr = RandomForestRegressor(n_jobs=-1,random_state=0)
grid = perform_grid_search(rfr, params, custom_scorer, X_train, y_train)


model = RandomForestRegressor(n_estimators = 500, 
                                criterion = "squared_error", # ["squared_error", "absolute_error", "poisson"],
                                max_depth = 1,               # [None, 1, 2, 3],
                                min_samples_split = 1.0,       # [1, 2, 3],
                                max_features = "sqrt",      # ["sqrt", "log2", None]
                                n_jobs=-1,
                                random_state=0)
model.fit(X_train, y_train)
model.predict(X_test)

# Évaluez le modèle en utilisant le scoreur personnalisé
score = custom_scorer(model, X_test, y_test)
print("\nRMSE Score: %.2f$" % score)
print("Score: %.2f%%" %model.score(X_test, y_test))

# Évaluez le modèle en utilisant le cross_val
rmse_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5,scoring=custom_scorer)
cv_regressor_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5)
print("\nCV RMSE Score : %.2f$" % np.mean(rmse_score))
print("CV Score : %.2f%%" % np.mean(cv_regressor_score))

grid.best_score_:  -73260650.02901699
grid.best_params_:  {'criterion': 'squared_error', 'max_depth': 1, 'max_features': 'sqrt', 'min_samples_split': 1.0, 'n_estimators': 500}

RMSE Score: -72645305.75$
Score: -0.00%

CV RMSE Score : -73005230.30$
CV Score : -0.00%


### GridSearch on KNN

In [29]:
## GridSearch
# Définir les valeurs à tester pour les hyperparamètres du modèle
params =    {'n_neighbors' : range(1, 50), 
            'weights' : ['uniform', 'distance']}
knn = KNeighborsRegressor(n_jobs=-1)
grid = perform_grid_search(knn, params, custom_scorer, X_train, y_train)

# CHECK LES RESULTATS
model = KNeighborsRegressor(n_neighbors= 43, weights= 'uniform', n_jobs=-1)
model.fit(X_train, y_train)
model.predict(X_test)

# Évaluez le modèle en utilisant le scoreur personnalisé
score = custom_scorer(model, X_test, y_test)
print("RMSE Score: %.2f$" % score)
print("Score: %.2f%%" %model.score(X_test, y_test))

# Évaluez le modèle en utilisant le cross_val
rmse_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5,scoring=custom_scorer)
knn_regressor_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5)
print("\nCV RMSE Score : %.2f$" % np.mean(rmse_score))
print("CV Score : %.2f%%" % np.mean(knn_regressor_score))

KeyboardInterrupt: 

### Grid on MLP

In [ ]:
from sklearn.neural_network import MLPRegressor

params =    {'hidden_layer_sizes' : [(50,50,50), (100,5), (100,)],
            'activation' : ['relu'],
            'solver': ['adam'],
            'alpha' : np.linspace(0.0001, 0.001, 10, endpoint = True),
            'learning_rate_init': [0.001, 0.01, 0.1],
            'max_iter': [500]}
model = MLPRegressor(random_state=0)

grid = perform_grid_search(model, params, custom_scorer, X_train, y_train)

### Training on MLP

In [ ]:
model = MLPRegressor(activation = 'relu', 
                    alpha = 0.0007, 
                    hidden_layer_sizes = (50, 50, 50), 
                    learning_rate_init = 0.1, 
                    max_iter = 500, 
                    solver = 'adam',
                    random_state=0)

model.fit(X_train, y_train)
model.predict(X_test)

# Évaluez le modèle en utilisant le scoreur personnalisé
score = custom_scorer(model, X_test, y_test)
print("RMSE Score: %.2f$" % score)
print("Score: %.2f%%" % model.score(X_test, y_test))

# Évaluez le modèle en utilisant le cross_val
rmse_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5,scoring=custom_scorer)
mlp_regressor_score = cross_val_score(model, preprocessed_data, Y1.values.ravel(), cv=5)
print("\nCV RMSE Score : %.2f$" % np.mean(rmse_score))
print("CV Score : %.2f%%" % np.mean(mlp_regressor_score))

In [ ]:
# Score for each training subset
N, train_score, val_score = learning_curve(model, X_train, y_train, cv=5)

plt.plot(N, train_score.mean(axis=1), label='train')
plt.plot(N, val_score.mean(axis=1), label='validation')
plt.xlabel("train size")
plt.legend()
plt.show()

In [ ]:
# Evaluation metrics from the gridsearch
res = pd.DataFrame(grid.cv_results_.values(), ).transpose()
res.columns=grid.cv_results_.keys()
res